# Цель работы: 
Научится анализировать текстовые данные(отзывы)

## Задача:
1)  Создайте свой «мешок слов», например, набор из 20-25 отзывов о чем-либо, напишите краткое пояснение к данным.
2)	Создайте словарь стоп-слов, примените стоп-слова к своему «мешку слов»
3)	Масштабируйте данные с помощью tf-idf
4)	Исследуйте коэффициенты модели
5)	Сделайте выводы по работе.

Я создал также папку coffee который содержит в общем 24 отзыва из которых 12 положительные и 12 отрицательные и все поделены на 2 группы(тестовые и обучающие)

In [4]:
from sklearn.datasets import load_files
reviews_train = load_files("coffee/train/")

text_train, y_train = reviews_train.data, reviews_train.target
print("тип text_train: {}".format(type(text_train)))
print("длина text_train: {}".format(len(text_train)))
print("text_train[1]:\n{}".format(text_train[1].decode('utf-8')))

тип text_train: <class 'list'>
длина text_train: 12
text_train[1]:
У них лучшие пироги, идеальное дополнение к напиткам!


In [5]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [7]:
print("text_train[1]:\n{}".format(text_train[1].decode('utf-8')))

text_train[1]:
У них лучшие пироги, идеальное дополнение к напиткам!


In [8]:
import numpy as np
np.unique(y_train)

array([0, 1])

In [9]:
print("Количество примеров на класс (обучение training): {}".format(np.bincount(y_train)))


Количество примеров на класс (обучение training): [6 6]


In [11]:
reviews_test = load_files("coffee/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("Количество документов в текстовых данных: [".format(len(text_test)))
print("Количество примеров на класс (тест - test): {}".format(np.bincount(y_test)))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

Количество документов в текстовых данных: [
Количество примеров на класс (тест - test): [6 6]


## Мешок слов

In [12]:
stop_words = ['и', 'в', 'на', 'с', 'не', 'что', 'за', 'это', 'по', 'к', 'для', 'от', 'но', 'из', 'о', 'а']

In [29]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Загрузка обучающего и тестового набора данных
reviews_train = load_files("coffee/train/")
reviews_test = load_files("coffee/test/")

text_train, y_train = reviews_train.data, reviews_train.target
text_test, y_test = reviews_test.data, reviews_test.target

In [31]:

filtered_text_train = [' '.join([word for word in review.decode('utf-8').split() if word.lower() not in stop_words]) for review in text_train]
filtered_text_test = [' '.join([word for word in review.decode('utf-8').split() if word.lower() not in stop_words]) for review in text_test]

In [32]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(text_train)
X_test = vectorizer.transform(text_test)

In [33]:
model = LogisticRegression()
scores = cross_val_score(model, X_train, y_train, cv=5)

In [34]:
print("Средняя правильность перекрестной проверки: {:.2f}".format(scores.mean()))

Средняя правильность перекрестной проверки: 0.60


In [35]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=100), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)

Наилучшее значение перекрестной проверки: 0.67
Наилучшие параметры:  {'C': 10}


## Масштабирование данных с помощью tf-idf

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.95)
X_train_tfidf = tfidf_vectorizer.fit_transform(text_train)

logistic_regression = LogisticRegression(max_iter=100)

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid_search.best_score_))


Наилучшее значение перекрестной проверки: 0.60


## Исследование коэффициентов модели

In [39]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)
coefficients = model.coef_

print("Коэффициенты модели LogisticRegression:")
print(coefficients)


Коэффициенты модели LogisticRegression:
[[ 0.15985859  0.15309597 -0.15972742  0.1346154   0.1480003  -0.15247693
  -0.15972742  0.24708987  0.15309597 -0.0215665   0.1346154   0.15985859
  -0.1369118  -0.15494641  0.15985859  0.15309597  0.1346154   0.1480003
  -0.15247693  0.15985859  0.1346154   0.15985859  0.1346154  -0.20675383
   0.23735901 -0.20834941  0.1480003   0.1346154   0.15985859  0.15985859
   0.15309597 -0.18108984  0.00053164  0.15985859 -0.15494641  0.15985859
   0.15985859  0.02491379 -0.31610866 -0.20675383 -0.26401874  0.15985859
  -0.15494641 -0.15972742  0.23735901  0.1480003   0.15985859 -0.18108984
  -0.18108984 -0.15247693  0.15309597 -0.15972742 -0.15247693 -0.15494641
  -0.20675383 -0.40990712 -0.20834941  0.1480003  -0.15494641 -0.15494641
   0.1480003  -0.11537367  0.1480003  -0.15972742  0.1346154 ]]


In [40]:
import numpy as np
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

top_positive_coefficients = np.argsort(coefficients)[0][-10:]  # Топ положительных коэффициентов
top_negative_coefficients = np.argsort(coefficients)[0][:10]  # Топ отрицательных коэффициентов

# Вывод топ слов с наибольшими положительными и отрицательными коэффициентами
print("Топ слов с наибольшими положительными коэффициентами:")
print(feature_names[top_positive_coefficients])

print("\nТоп слов с наименьшими отрицательными коэффициентами:")
print(feature_names[top_negative_coefficients])


Топ слов с наибольшими положительными коэффициентами:
['них' 'напиткам' 'найти' 'лучшие' 'можно' 'здесь' 'американо' 'оправданы'
 'качеством' 'всегда']

Топ слов с наименьшими отрицательными коэффициентами:
['слишком' 'не' 'неудобно' 'качеству' 'соответствуют' 'неоригинальный'
 'интерьер' 'скучный' 'маленькие' 'пирогов']


## Вывод и итог:

 Наш анализ данных, применение моделей и исследование коэффициентов позволили получить ценные данные из текстовых данных об отзывах о кофейне.
  Мы сделали предобработки данных, загрузив отзывы, фильтруя стоп-слова и использовав TF-IDF для создания числовых признаков из текста.
 Благодаря такому методу мы можем оценивать отзывы о любых вещях, заведениях и тп. Использовать эту модель для анализа тексттовых данных.